In [19]:
import paramiko
from getpass import getpass
import os

# Check to see if bodyFileResults.txt file exists
if os.path.exists('bodyFileResults.txt'):
    # If so ask user to delete file
    user_input = input("That file exists do you want to delete the file?(Y/N)")

    # If yes delete the file
    if user_input == "Y":
        os.remove('bodyFileResults.txt')

# create the password prompt
thePass = getpass(prompt="Please enter your SSH password")

# Host information
host = "192.168.1.17"
port = 22
username = "emily"
password = thePass

# try connecting via ssh using paramiko
try :
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host,port,username,password)
# if fails from authentication error print that it failed due to authentication failure
except paramiko.AuthenticationException:
    print ("Authentication failed")

In [20]:
# start SFTP session
sftp = ssh.open_sftp()

# the local file on the system to upload
localFile = "fs.py"
# the remote path to upload the file to
remotePath = "/home/emily/fs.py"

# push the file to the linux server in the correct path
sftp.put(localFile, remotePath)

# Run the file that was pushed
command = "python3 fs.py -d /usr/bin"

# Get the output from the command
stdin, stdout, stderr = ssh.exec_command(command)

# Get results from stout
lines = stdout.readlines()

# Convert the list to a string
output = ''.join(lines)
# Header output
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'
# Footer
sepFooter = '' + '### END ' + command  + ' ###\n\n'
# Concatenates the header, command, footer
cmd_output = sepHeader + output + sepFooter

# save the cmd_output to a file
with open('bodyFileResults.txt', 'a') as f:
    f.write(cmd_output)


In [21]:
# close the ssh session
ssh.close()